In [61]:
import sys    
sys.path.insert(0, '/Users/niyi/Documents/GitHub/Optimal-Control/Tools')
from ActorCriticNetworks import ActorNetwork, CriticNetwork
import torch as T
import torch.nn as nn 
import os
project_path= os.path.dirname(os.path.abspath(os.curdir))

In [62]:
test_state= T.rand((5, 4))
test_action= T.rand((5, 2))

In [63]:
test_critic1= CriticNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_critic1', learning_rate=0.02, project_path=project_path)
test_critic1.save_checkpoint()
print(test_critic1(test_state, test_action))
test_critic1.load_checkpoint()
print(test_critic1(test_state, test_action))

... saving checkpoint ...
tensor([[-0.1627],
        [-0.5450],
        [-0.3962],
        [-0.6051],
        [-0.5436]], grad_fn=<AddmmBackward0>)
... loading checkpoint ...
tensor([[-0.1627],
        [-0.5450],
        [-0.3962],
        [-0.6051],
        [-0.5436]], grad_fn=<AddmmBackward0>)


In [64]:
test_actor1= ActorNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), 'test_actor', 0.02, project_path=project_path)
test_actor1.save_checkpoint()
print(test_actor1(test_state))
test_actor1.load_checkpoint()
print(test_actor1(test_state))

... saving checkpoint ...
tensor([[ 0.5998,  0.2954],
        [ 0.5187,  0.1982],
        [ 0.5648,  0.2736],
        [ 0.9089,  0.0211],
        [ 0.9021, -0.0899]], grad_fn=<AddmmBackward0>)
... loading checkpoint ...
tensor([[ 0.5998,  0.2954],
        [ 0.5187,  0.1982],
        [ 0.5648,  0.2736],
        [ 0.9089,  0.0211],
        [ 0.9021, -0.0899]], grad_fn=<AddmmBackward0>)


**3. Functional tests** to check if this module functions as intended.They only verify the output of an action and do not check the intermediate states of the system when performing that action. 

Can copy parameters

In [65]:
test_critic2= CriticNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_critic2', learning_rate=0.02, project_path=project_path)

print(test_critic2(test_state, test_action))
for target_param, param in zip(test_critic2.parameters(), test_critic1.parameters()):
    target_param.data.copy_(param.data)
print(test_critic2(test_state, test_action))

tensor([[0.5238],
        [0.3752],
        [0.4954],
        [0.4982],
        [0.5368]], grad_fn=<AddmmBackward0>)
tensor([[-0.1627],
        [-0.5450],
        [-0.3962],
        [-0.6051],
        [-0.5436]], grad_fn=<AddmmBackward0>)


In [66]:
test_actor2= ActorNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_actor2', learning_rate=0.02, project_path=project_path)
print(test_actor2(test_state))
for target_param, param in zip(test_actor2.parameters(), test_actor1.parameters()):
    target_param.data.copy_(param.data)
print(test_actor2(test_state))

tensor([[ 0.1522, -0.2019],
        [ 0.0952,  0.1890],
        [ 0.1467,  0.0804],
        [ 0.0871, -0.1662],
        [ 0.1609, -0.2308]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5998,  0.2954],
        [ 0.5187,  0.1982],
        [ 0.5648,  0.2736],
        [ 0.9089,  0.0211],
        [ 0.9021, -0.0899]], grad_fn=<AddmmBackward0>)


Can learn to approximate values

In [67]:
y_train= T.rand((5, 1))
epochs = 300
for epoch in range(epochs):
    y_pred = test_critic1(test_state, test_action)
    loss = T.nn.MSELoss()(y_pred, y_train)
    test_critic1.optimizer.zero_grad()
    loss.backward()
    test_critic1.optimizer.step()

test_critic1(test_state, test_action), y_train

(tensor([[0.7515],
         [0.8123],
         [0.4007],
         [0.2289],
         [0.4435]], grad_fn=<AddmmBackward0>),
 tensor([[0.7515],
         [0.8123],
         [0.4007],
         [0.2289],
         [0.4435]]))

In [68]:
epochs = 300
for epoch in range(epochs):
    y_pred = test_actor1(test_state)
    loss = T.nn.MSELoss()(y_pred, test_action)
    test_actor1.optimizer.zero_grad()
    loss.backward()
    test_actor1.optimizer.step()

test_actor1(test_state), test_action

(tensor([[0.0381, 0.9804],
         [0.9591, 0.0814],
         [0.4551, 0.9332],
         [0.8314, 0.5219],
         [0.3607, 0.7818]], grad_fn=<AddmmBackward0>),
 tensor([[0.0381, 0.9804],
         [0.9591, 0.0814],
         [0.4551, 0.9332],
         [0.8314, 0.5219],
         [0.3607, 0.7818]]))